In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
from datetime import datetime
from sklearn.metrics import recall_score

In [187]:
pip install xgboost --user --upgrade pip

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Requirement already up-to-date: xgboost in /root/.local/lib/python3.6/site-packages (1.1.1)
Requirement already up-to-date: pip in /root/.local/lib/python3.6/site-packages (20.1.1)


In [301]:
tr_orig=pd.read_csv('Train.csv')
te_orig=pd.read_csv('Test.csv')
sample=pd.read_csv('sample_submission.csv')

In [302]:
tr_orig.head()

,INCIDENT_ID,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,MULTIPLE_OFFENSE
0,CR_102659,04-JUL-04,0,36,34,2,1,5,6,1,6,1,174,1.0,92,29,36,0
1,CR_189752,18-JUL-17,1,37,37,0,0,11,17,1,6,1,236,1.0,103,142,34,1
2,CR_184637,15-MAR-17,0,3,2,3,5,1,0,2,3,1,174,1.0,110,93,34,1
3,CR_139071,13-FEB-09,0,33,32,2,1,7,1,1,6,1,249,1.0,72,29,34,1
4,CR_109335,13-APR-05,0,33,32,2,1,8,3,0,5,1,174,0.0,112,29,43,1


In [303]:
tr_orig.shape, te_orig.shape

((23856, 18), (15903, 17))

In [304]:
te_orig.head()

,INCIDENT_ID,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,CR_195453,01-FEB-18,0,30,35,7,3,6,4,0,5,1,174,NaN,72,119,23
1,CR_103520,05-MAR-04,0,44,44,1,3,7,1,4,6,1,316,0.0,12,29,34
2,CR_196089,27-JAN-18,0,34,33,3,5,2,7,3,0,1,316,1.0,72,0,34
3,CR_112195,18-AUG-06,7,3,2,3,5,9,8,0,5,1,174,1.0,112,87,34
4,CR_149832,31-OCT-11,0,7,8,7,3,2,7,1,5,1,174,0.0,112,93,43


In [305]:
sample.head()

,INCIDENT_ID,MULTIPLE_OFFENSE
0,CR_195453,1
1,CR_103520,0
2,.,.
3,.,.
4,CR_196089,1


In [306]:
same=te_orig[te_orig.INCIDENT_ID.isin(list(set(tr_orig.INCIDENT_ID)))]

In [307]:
same

,INCIDENT_ID,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15


In [308]:
tr_orig['DATE'].dtype, te_orig['DATE'].dtype

(dtype('O'), dtype('O'))

In [309]:
tr_orig['DATE']= pd.to_datetime(tr_orig['DATE'])
te_orig['DATE']= pd.to_datetime(te_orig['DATE'])

In [310]:
tr_orig['DATE'].dtype, te_orig['DATE'].dtype

(dtype('<M8[ns]'), dtype('<M8[ns]'))

In [311]:
tr_orig.describe()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,MULTIPLE_OFFENSE
count,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23674.000000,23856.000000,23856.000000,23856.000000,23856.000000
mean,0.483778,24.791206,24.637450,4.276744,2.455609,6.154175,4.876509,0.972460,4.924128,1.244802,206.954519,0.974064,85.237383,72.674296,33.464747,0.955231
std,1.439738,15.240231,15.135093,2.944672,1.963095,4.471756,3.881931,1.453144,1.362625,1.119301,93.033348,1.167725,27.597226,43.297320,8.386834,0.206800
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,7.000000,8.000000,2.000000,1.000000,3.000000,2.000000,0.000000,5.000000,1.000000,174.000000,1.000000,72.000000,29.000000,34.000000,1.000000
50%,0.000000,24.000000,24.000000,4.000000,3.000000,5.000000,4.000000,1.000000,5.000000,1.000000,249.000000,1.000000,98.000000,62.000000,34.000000,1.000000
75%,0.000000,36.000000,35.000000,6.000000,5.000000,8.000000,7.000000,1.000000,6.000000,1.000000,249.000000,1.000000,103.000000,107.000000,34.000000,1.000000
max,7.000000,52.000000,52.000000,10.000000,5.000000,19.000000,18.000000,99.000000,6.000000,90.000000,332.000000,90.000000,116.000000,142.000000,50.000000,1.000000


In [312]:
target=tr_orig['MULTIPLE_OFFENSE']
tr_id=tr_orig['INCIDENT_ID']
te_id=te_orig['INCIDENT_ID']

In [313]:
train=tr_orig.copy()
test=te_orig.copy()
tr_orig.drop('MULTIPLE_OFFENSE', axis=1, inplace=True)
tr_orig.drop('INCIDENT_ID', axis=1, inplace=True)
te_orig.drop('INCIDENT_ID', axis=1, inplace=True)

In [314]:
tr_orig.head()

,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,2004-07-04,0,36,34,2,1,5,6,1,6,1,174,1.0,92,29,36
1,2017-07-18,1,37,37,0,0,11,17,1,6,1,236,1.0,103,142,34
2,2017-03-15,0,3,2,3,5,1,0,2,3,1,174,1.0,110,93,34
3,2009-02-13,0,33,32,2,1,7,1,1,6,1,249,1.0,72,29,34
4,2005-04-13,0,33,32,2,1,8,3,0,5,1,174,0.0,112,29,43


In [315]:
te_orig.head()

,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15
0,2018-02-01,0,30,35,7,3,6,4,0,5,1,174,NaN,72,119,23
1,2004-03-05,0,44,44,1,3,7,1,4,6,1,316,0.0,12,29,34
2,2018-01-27,0,34,33,3,5,2,7,3,0,1,316,1.0,72,0,34
3,2006-08-18,7,3,2,3,5,9,8,0,5,1,174,1.0,112,87,34
4,2011-10-31,0,7,8,7,3,2,7,1,5,1,174,0.0,112,93,43


In [316]:
tr_orig['week_number_of_year'] = tr_orig['DATE'].dt.week
tr_orig['month_number_of_year'] = tr_orig['DATE'].dt.month
tr_orig['year'] = tr_orig['DATE'].dt.year




In [317]:
te_orig['week_number_of_year'] = te_orig['DATE'].dt.week
te_orig['month_number_of_year'] = te_orig['DATE'].dt.month
te_orig['year'] = te_orig['DATE'].dt.year

In [318]:
tr_orig.head()

,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,week_number_of_year,month_number_of_year,year
0,2004-07-04,0,36,34,2,1,5,6,1,6,1,174,1.0,92,29,36,27,7,2004
1,2017-07-18,1,37,37,0,0,11,17,1,6,1,236,1.0,103,142,34,29,7,2017
2,2017-03-15,0,3,2,3,5,1,0,2,3,1,174,1.0,110,93,34,11,3,2017
3,2009-02-13,0,33,32,2,1,7,1,1,6,1,249,1.0,72,29,34,7,2,2009
4,2005-04-13,0,33,32,2,1,8,3,0,5,1,174,0.0,112,29,43,15,4,2005


In [319]:
te_orig.head()

,DATE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,week_number_of_year,month_number_of_year,year
0,2018-02-01,0,30,35,7,3,6,4,0,5,1,174,NaN,72,119,23,5,2,2018
1,2004-03-05,0,44,44,1,3,7,1,4,6,1,316,0.0,12,29,34,10,3,2004
2,2018-01-27,0,34,33,3,5,2,7,3,0,1,316,1.0,72,0,34,4,1,2018
3,2006-08-18,7,3,2,3,5,9,8,0,5,1,174,1.0,112,87,34,33,8,2006
4,2011-10-31,0,7,8,7,3,2,7,1,5,1,174,0.0,112,93,43,44,10,2011


In [320]:
tr_orig['month_stddev']=(tr_orig['month_number_of_year']-tr_orig['month_number_of_year'].mean())/tr_orig['month_number_of_year'].mean()
tr_orig['week_stddev']=(tr_orig['week_number_of_year']-tr_orig['week_number_of_year'].mean())/tr_orig['week_number_of_year'].mean()

te_orig['month_stddev']=(te_orig['month_number_of_year']-te_orig['month_number_of_year'].mean())/te_orig['month_number_of_year'].mean()
te_orig['week_stddev']=(te_orig['week_number_of_year']-te_orig['week_number_of_year'].mean())/te_orig['week_number_of_year'].mean()

In [321]:
tr_orig['X_12'].fillna(tr_orig['X_12'].median(), inplace=True)
te_orig['X_12'].fillna(te_orig['X_12'].median(), inplace=True)

In [322]:
tr_orig.isnull().sum(), te_orig.isnull().sum()

(DATE                    0
 X_1                     0
 X_2                     0
 X_3                     0
 X_4                     0
 X_5                     0
 X_6                     0
 X_7                     0
 X_8                     0
 X_9                     0
 X_10                    0
 X_11                    0
 X_12                    0
 X_13                    0
 X_14                    0
 X_15                    0
 week_number_of_year     0
 month_number_of_year    0
 year                    0
 month_stddev            0
 week_stddev             0
 dtype: int64, DATE                    0
 X_1                     0
 X_2                     0
 X_3                     0
 X_4                     0
 X_5                     0
 X_6                     0
 X_7                     0
 X_8                     0
 X_9                     0
 X_10                    0
 X_11                    0
 X_12                    0
 X_13                    0
 X_14                    0
 X_15         

In [323]:
for x in (tr_orig.columns):
  print(x+":")
  print(tr_orig[x].unique())

DATE:
['2004-07-04T00:00:00.000000000' '2017-07-18T00:00:00.000000000'
 '2017-03-15T00:00:00.000000000' ... '2014-09-14T00:00:00.000000000'
 '2014-05-01T00:00:00.000000000' '1994-11-08T00:00:00.000000000']
X_1:
[0 1 5 7 3 4 2 6]
X_2:
[36 37  3 33 45 30  8 49  4  7 25 21 47 22 15 16 11 48 39  5 38 46 19 18
 35  9  2 44 24 40 17 42 41 43 23  6 28 31 29 14 50 26 34 20 13 32 51  0
 27  1 52 10]
X_3:
[34 37  2 32 45 35  7 49  4  8 25 23 48 22 14 15 11 47 39  5 38 46 19 18
 30 10  3 44 24 40 16 42 41 43 21  6 28 36 29 17 51 27 33 20 13 31 50  1
 26  0 52  9]
X_4:
[ 2  0  3 10  7  6  9  4  1  5]
X_5:
[1 0 5 3 2]
X_6:
[ 5 11  1  7  8  9 15  2  4 13  3 16  6 18 12 17 14 10 19]
X_7:
[ 6 17  0  1  3  8 10  7  2 18  5 12  4 13 16 11 14 15  9]
X_8:
[ 1  2  0  3  4  5  7  6 12 10  8 20 11 15  9 99 14 13 30 16 21 29 50 22]
X_9:
[6 3 5 1 2 0 4]
X_10:
[ 1  2  3  4  6 12  5 11 10 90 20  8 16 30 19  7 50  9 18 58 22 40 15 17]
X_11:
[174 236 249 303 316 145   0 127 299 158  12  74 328  60 263 162 231 179


In [324]:
for x in (tr_orig.columns):
  print(x+":")
  print(tr_orig[x].nunique())

DATE:
9121
X_1:
8
X_2:
52
X_3:
52
X_4:
10
X_5:
5
X_6:
19
X_7:
19
X_8:
24
X_9:
7
X_10:
24
X_11:
133
X_12:
23
X_13:
60
X_14:
62
X_15:
28
week_number_of_year:
53
month_number_of_year:
12
year:
28
month_stddev:
12
week_stddev:
53


In [325]:
today=pd.to_datetime('today')
today


Timestamp('2020-06-21 16:10:13.982745')

In [326]:
tr_orig['nb_days'] = ((today- tr_orig.DATE)/np.timedelta64(1, 'D'))
tr_orig['nb_months'] = ((today- tr_orig.DATE)/np.timedelta64(1, 'M'))
tr_orig['nb_weeks'] = ((today- tr_orig.DATE)/np.timedelta64(1, 'W'))
tr_orig['nb_years'] = ((today- tr_orig.DATE)/np.timedelta64(1, 'Y'))

te_orig['nb_days'] = ((today- te_orig.DATE)/np.timedelta64(1, 'D'))
te_orig['nb_months'] = ((today- te_orig.DATE)/np.timedelta64(1, 'M'))
te_orig['nb_weeks'] = ((today- te_orig.DATE)/np.timedelta64(1, 'W'))
te_orig['nb_years'] = ((today- te_orig.DATE)/np.timedelta64(1, 'Y'))


In [327]:
tr_orig['X11_stddev']=(tr_orig['X_11']-tr_orig['X_11'].mean())/tr_orig['X_11'].mean()
#tr_orig['X12_stddev']=(tr_orig['X_12']-tr_orig['X_12'].mean())/tr_orig['X_12'].mean()
#tr_orig['X13_stddev']=(tr_orig['X_13']-tr_orig['X_13'].mean())/tr_orig['X_13'].mean()
#tr_orig['X14_stddev']=(tr_orig['X_14']-tr_orig['X_14'].mean())/tr_orig['X_14'].mean()
tr_orig['X2_ratio']=tr_orig['X_2']/tr_orig['X_2'].max()
tr_orig['X3_ratio']=tr_orig['X_3']/tr_orig['X_3'].max()
tr_orig['X10_ratio']=tr_orig['X_10']/tr_orig['X_10'].max()
tr_orig['X11_ratio']=tr_orig['X_11']/tr_orig['X_11'].max()
tr_orig['X12_ratio']=tr_orig['X_12']/tr_orig['X_12'].max()
tr_orig['X13_ratio']=tr_orig['X_13']/tr_orig['X_13'].max()
tr_orig['X14_ratio']=tr_orig['X_14']/tr_orig['X_14'].max()
tr_orig['X15_ratio']=tr_orig['X_15']/tr_orig['X_15'].max()


te_orig['X11_stddev']=(te_orig['X_11']-te_orig['X_11'].mean())/te_orig['X_11'].mean()
#te_orig['X12_stddev']=(te_orig['X_12']-te_orig['X_12'].mean())/te_orig['X_12'].mean()
#te_orig['X13_stddev']=(te_orig['X_13']-te_orig['X_13'].mean())/te_orig['X_13'].mean()
#te_orig['X14_stddev']=(te_orig['X_14']-te_orig['X_14'].mean())/te_orig['X_14'].mean()
te_orig['X2_ratio']=te_orig['X_2']/te_orig['X_2'].max()
te_orig['X3_ratio']=te_orig['X_3']/te_orig['X_3'].max()
te_orig['X10_ratio']=te_orig['X_10']/te_orig['X_10'].max()
te_orig['X11_ratio']=te_orig['X_11']/te_orig['X_11'].max()
te_orig['X12_ratio']=te_orig['X_12']/te_orig['X_12'].max()
te_orig['X13_ratio']=te_orig['X_13']/te_orig['X_13'].max()
te_orig['X14_ratio']=te_orig['X_14']/te_orig['X_14'].max()
te_orig['X15_ratio']=te_orig['X_15']/te_orig['X_15'].max()


In [328]:
#tr_orig['X2_X3_ratio'].replace(np.inf, -1, inplace=True)
#tr_orig['X6_X7_ratio'].replace(np.inf, -1, inplace=True)

#te_orig['X2_X3_ratio'].replace(np.inf,-1, inplace=True)
#te_orig['X6_X7_ratio'].replace(np.inf, -1, inplace=True)

In [329]:
train=tr_orig.copy()
test=te_orig.copy()
train.drop('DATE', axis=1, inplace=True)
test.drop('DATE', axis=1, inplace=True)

In [330]:
train.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,week_number_of_year,month_number_of_year,year,month_stddev,week_stddev,nb_days,nb_months,nb_weeks,nb_years,X11_stddev,X2_ratio,X3_ratio,X10_ratio,X11_ratio,X12_ratio,X13_ratio,X14_ratio,X15_ratio
0,0,36,34,2,1,5,6,1,6,1,174,1.0,92,29,36,27,7,2004,0.075460,0.016105,5831.673773,191.598966,833.096253,15.966580,-0.159236,0.692308,0.653846,0.011111,0.524096,0.011111,0.793103,0.204225,0.72
1,1,37,37,0,0,11,17,1,6,1,236,1.0,103,142,34,29,7,2017,0.075460,0.091372,1069.673773,35.144008,152.810539,2.928667,0.140347,0.711538,0.711538,0.011111,0.710843,0.011111,0.887931,1.000000,0.68
2,0,3,2,3,5,1,0,2,3,1,174,1.0,110,93,34,11,3,2017,-0.539089,-0.586031,1194.673773,39.250868,170.667682,3.270906,-0.159236,0.057692,0.038462,0.011111,0.524096,0.011111,0.948276,0.654930,0.68
3,0,33,32,2,1,7,1,1,6,1,249,1.0,72,29,34,7,2,2009,-0.692726,-0.736565,4146.673773,136.238486,592.381968,11.353207,0.203163,0.634615,0.615385,0.011111,0.750000,0.011111,0.620690,0.204225,0.68
4,0,33,32,2,1,8,3,0,5,1,174,0.0,112,29,43,15,4,2005,-0.385452,-0.435497,5548.673773,182.301034,792.667682,15.191753,-0.159236,0.634615,0.615385,0.011111,0.524096,0.000000,0.965517,0.204225,0.86


In [331]:
train.describe()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,week_number_of_year,month_number_of_year,year,month_stddev,week_stddev,nb_days,nb_months,nb_weeks,nb_years,X11_stddev,X2_ratio,X3_ratio,X10_ratio,X11_ratio,X12_ratio,X13_ratio,X14_ratio,X15_ratio
count,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,2.385600e+04,2.385600e+04,23856.000000,23856.000000,23856.000000,23856.000000,2.385600e+04,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000,23856.000000
mean,0.483778,24.791206,24.637450,4.276744,2.455609,6.154175,4.876509,0.972460,4.924128,1.244802,206.954519,0.974262,85.237383,72.674296,33.464747,26.572057,6.508845,2004.249120,1.801041e-18,-2.364157e-17,5743.788713,188.711512,820.541245,15.725959,5.020111e-17,0.476754,0.473797,0.013831,0.623357,0.010825,0.734805,0.511791,0.669295
std,1.439738,15.240231,15.135093,2.944672,1.963095,4.471756,3.881931,1.453144,1.362625,1.119301,93.033348,1.163264,27.597226,43.297320,8.386834,14.372609,3.293803,7.795184,5.060503e-01,5.408918e-01,2850.551598,93.654542,407.221657,7.804545,4.495352e-01,0.293081,0.291059,0.012437,0.280221,0.012925,0.237907,0.304911,0.167737
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1991.000000,-8.463629e-01,-9.623665e-01,538.673773,17.698064,76.953396,1.474839,-1.000000e+00,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,7.000000,8.000000,2.000000,1.000000,3.000000,2.000000,0.000000,5.000000,1.000000,174.000000,1.000000,72.000000,29.000000,34.000000,15.000000,4.000000,1998.000000,-3.854516e-01,-4.354972e-01,3368.673773,110.677386,481.239110,9.223116,-1.592356e-01,0.134615,0.153846,0.011111,0.524096,0.011111,0.620690,0.204225,0.680000
50%,0.000000,24.000000,24.000000,4.000000,3.000000,5.000000,4.000000,1.000000,5.000000,1.000000,249.000000,1.000000,98.000000,62.000000,34.000000,27.000000,7.000000,2004.000000,7.545967e-02,1.610499e-02,5870.673773,192.880306,838.667682,16.073359,2.031629e-01,0.461538,0.461538,0.011111,0.750000,0.011111,0.844828,0.436620,0.680000
75%,0.000000,36.000000,35.000000,6.000000,5.000000,8.000000,7.000000,1.000000,6.000000,1.000000,249.000000,1.000000,103.000000,107.000000,34.000000,39.000000,9.000000,2011.000000,3.827339e-01,4.677072e-01,8151.673773,267.822297,1164.524825,22.318525,2.031629e-01,0.692308,0.673077,0.011111,0.750000,0.011111,0.887931,0.753521,0.680000
max,7.000000,52.000000,52.000000,10.000000,5.000000,19.000000,18.000000,99.000000,6.000000,90.000000,332.000000,90.000000,116.000000,142.000000,50.000000,53.000000,12.000000,2018.000000,8.436451e-01,9.945765e-01,10764.673773,353.672109,1537.810539,29.472676,6.042172e-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [332]:
#####################################################################################################

In [333]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.25, random_state=42)

In [334]:
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split


In [335]:
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV  #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [346]:
predictors = [x for x in train.columns]
xgb = XGBClassifier(
 learning_rate =0.1,
 n_estimators=800,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=800, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [347]:
pred = xgb.predict(X_test)


In [348]:
recall_score(y_test, pred, average='weighted')

0.9994969818913481

In [349]:
predictors = [x for x in train.columns]
xgb = XGBClassifier(
 learning_rate =0.1,
 n_estimators=800,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
xgb.fit(train, target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=800, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [350]:
pred_test = xgb.predict(test)

In [351]:
outdf1=pd.DataFrame({'INCIDENT_ID': te_id, 'MULTIPLE_OFFENSE': pred_test})

In [352]:
outdf1.head(), outdf1.shape

(  INCIDENT_ID  MULTIPLE_OFFENSE
 0   CR_195453                 1
 1   CR_103520                 1
 2   CR_196089                 1
 3   CR_112195                 1
 4   CR_149832                 1, (15903, 2))

In [353]:
outdf1['MULTIPLE_OFFENSE'].unique()

array([1, 0])

In [354]:
outdf1.to_csv('Submission6.csv', index=False)